In [ ]:
import numpy as np
from pylops.waveeqprocessing.mdd import MDC

import time

In [ ]:
inputfile = './data/pniose_for_cc_stacking.npz'
inputdata = np.load(inputfile)

pfull= inputdata['pfull']
p_without_direct = inputdata['p_without_direct']

# Parameters
split_size = 1500
total_size = pfull.shape[-1]  

# Split the matrix
for i in range(61):
    # For the first 60 matrices with shape (351, 401, 1500)
    submatrix = pfull[:, :, i*split_size:(i+1)*split_size]
    submatrix = submatrix.astype(np.float32)
    filename = f"data/pfull_{i}.npy"
    np.save(filename, submatrix)
    print(f"Saved {filename} with shape {submatrix.shape}")

for i in range(61):
    # For the first 60 matrices with shape (351, 401, 1500)
    submatrix = p_without_direct[:, :, i*split_size:(i+1)*split_size]
    submatrix = submatrix.astype(np.float32)
    filename = f"data/pwithoutdir_{i}.npy"
    np.save(filename, submatrix)
    print(f"Saved {filename} with shape {submatrix.shape}")    

In [ ]:
# Start the timer
start_time = time.time()

twosided=True
add_negative=True
saveGt=True
fftengine = "numpy"
nt = split_size
nt2 = nt*2-1
nr = 400
dt = 0.002
dr = 4

GHd_sum = np.zeros((nr, nr, nt2))
GHG_sum = np.zeros((nr, nr, nt2))


In [ ]:
# Load and sum matrices from pfull_0.npy to pfull_59.npy
for i in range(60):
    print('start to run: ', i)
    
    # Load the current matrix
    G = np.load(f'data/pfull_{i}.npy')
    d = np.load(f'data/pwithoutdir_{i}.npy')
    
    ns, nr, nt = G.shape
    if len(d.shape) == 2:
        nv = 1
    else:
        nv = d.shape[1]
    
    nt2 = 2 * nt - 1
    nfmax_allowed = int(np.ceil((nt2 + 1) / 2))

    # Add negative part to data and model
    d = np.concatenate((np.squeeze(np.zeros((ns, nv, nt - 1))), d), axis=-1)
    G1 = np.concatenate((np.squeeze(np.zeros((ns, nr, nt - 1))), G), axis=-1)
    
    G = np.concatenate((np.zeros((ns, nr, nt - 1)), G), axis=-1)
    
    # Bring kernel to frequency domain
    Gfft = np.fft.rfft(G, nt2, axis=-1)
    
    # Bring frequency/time to first dimension
    Gfft = np.moveaxis(Gfft, -1, 0)
    d = np.moveaxis(d, -1, 0)
    G1 = np.moveaxis(G1, -1, 0)
    
    # Define MDC linear operator
    MDCop = MDC(
        Gfft,
        nt2,
        nv=nr,
        dt=dt,
        dr=dr,
        twosided=twosided,
        saveGt=saveGt,
        fftengine=fftengine,
    )
    # Adjoint
    GHd = MDCop.H * d.ravel()
    GHd = np.squeeze(GHd.reshape(nt2, nr, nr))
    GHd = np.moveaxis(GHd, 0, -1)
    
    GHG = MDCop.H * G1.ravel()
    GHG = np.squeeze(GHG.reshape(nt2, nr, nr))
    GHG = np.moveaxis(GHG, 0, -1)
    
    GHd_sum = GHd_sum + GHd
    GHG_sum = GHG_sum + GHG
    
    print('finished: ', i)
    

In [ ]:
np.save('data/GHd_sum', GHd_sum)
np.save('data/GHG_sum', GHG_sum)

In [ ]:
# End the timer
end_time = time.time()

# Calculate and print the running time in minutes
running_time_mins = (end_time - start_time) / 60
print(f"Running time: {running_time_mins:.2f} minutes")
